In [1]:
!pip install -q mlflow paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, Subset
from random import sample
from torch.utils.data import DataLoader
from transformers import RobertaModel, RobertaTokenizer
from sklearn.utils import resample
from torchmetrics.classification import (
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm
import mlflow
import time
import pandas as pd
import os
import paramiko

2025-07-29 12:08:14.677866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753790894.849874      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753790894.902374      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import sys
sys.path.append('/kaggle/input/liar-plus-utils')
from utils import (
    LABEL_MAPPING,
    ids2labels,
    save_checkpoint,
    load_checkpoint,
    save_best_model,
    load_best_model,
    save_model_remotely
)

In [4]:
class LiarPlusSingleRobertaDataset(Dataset):
    def __init__(
        self,
        filepath: str,
        tokenizer,
        str_metadata_cols: list[str],
        num_metadata_cols: list[str],
        max_length: int = 512,
    ):
        self.df = pd.read_csv(filepath)

        self.str_metadata_cols = str_metadata_cols
        self.num_metadata_cols = num_metadata_cols

        for column in self.str_metadata_cols:
            self.df[column] = self.df[column].astype(str)

        self.df["statement"] = self.df["statement"].astype(str)
        self.df["justification"] = self.df["justification"].astype(str)

        self.statement_max_len = max_length // 4
        self.justification_max_len = max_length // 4
        self.str_metadata_max_len = (
            max_length - self.statement_max_len - self.justification_max_len
        ) // len(str_metadata_cols)

        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df.index)

    def limit_tokens(self, text, max_length=512):
        tokenized = self.tokenizer.tokenize(text)[:max_length]
        return self.tokenizer.decode(
            self.tokenizer.convert_tokens_to_ids(tokenized)
        )

    def __getitem__(self, index: int):
        item = self.df.iloc[index]

        input = self.limit_tokens(
            f"[STATEMENT] {item['statement']} ", self.statement_max_len
        )
        input += self.limit_tokens(
            f" [JUSTIFICATION] {item['justification']}",
            self.justification_max_len,
        )

        for column in self.str_metadata_cols:
            input += self.limit_tokens(
                f" [{column.upper()}] {item[column]}",
                self.str_metadata_max_len,
            )

        encoded = self.tokenizer(
            input,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        label = LABEL_MAPPING[item["label"]]

        num_metadata = [item[column] for column in self.num_metadata_cols]

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "num_metadata": torch.tensor(num_metadata).float(),
            "label": torch.tensor(label),
        }

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class LiarPlusSingleRoBERTasClassifier(nn.Module):
    def __init__(
        self, encoder_model, num_metadata_len, num_hidden, num_classes
    ):
        super(LiarPlusSingleRoBERTasClassifier, self).__init__()
        self.encoder = encoder_model
        self.hl = nn.Linear(
            self.encoder.config.hidden_size + num_metadata_len, num_hidden
        )
        self.dropout = nn.Dropout(p=0.1)
        self.fc = nn.Linear(num_hidden, num_classes)

    def forward(self, input_ids, attention_mask, num_metadata):
        with torch.no_grad():
            outputs = self.encoder(
                input_ids=input_ids, attention_mask=attention_mask
            )

        cls_embedding = outputs.last_hidden_state[:, 0, :]
        concatted_inputs = torch.cat([cls_embedding, num_metadata], dim=1)

        hl_output = F.gelu(self.hl(concatted_inputs))
        hl_output = self.dropout(hl_output)

        logits = self.fc(hl_output)
        return logits

    # Zapisz tylko wagi warstw klasyfikatora
    def state_for_save(self):
        return {
            'hl_state_dict': self.hl.state_dict(),
            'fc_state_dict': self.fc.state_dict(),
        }
        
    # Ładowanie modelu (tylko wagi klasyfikatora)
    def load_state_from_save(self, state):
        self.hl.load_state_dict(state['hl_state_dict'])
        self.fc.load_state_dict(state['fc_state_dict'])

In [6]:
def test(
    model: nn.Module,
    best_model_path: str,
    dataloader: DataLoader
) -> None:
    # Define loss function
    criterion = nn.CrossEntropyLoss()

    load_best_model(model, best_model_path)
    
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    f1 = MulticlassF1Score(num_classes, average=None).to(device)
    precision = MulticlassPrecision(num_classes, average=None).to(device)
    recall = MulticlassRecall(num_classes, average=None).to(device)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            num_metadata = batch["num_metadata"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask, num_metadata)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * input_ids.size(0)

            preds = torch.argmax(outputs, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += input_ids.size(0)

            f1.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    f1_res = f1.compute()
    precision_res = precision.compute()
    recall_res = recall.compute()

    mlflow.log_metric("test_acc", accuracy)
    mlflow.log_metric("test_loss", accuracy)

    for i in range(num_classes):
        mlflow.log_metric(f"test_f1_{ids2labels[i]}", f1_res[i])
        mlflow.log_metric(f"test_precision_{ids2labels[i]}", precision_res[i])
        mlflow.log_metric(f"test_recall_{ids2labels[i]}", recall_res[i])
    
    macro_f1 = f1_res.mean()
    macro_precision = precision_res.mean()
    macro_recall = recall_res.mean()

    mlflow.log_metric("test_f1", macro_f1)
    mlflow.log_metric("test_precision", macro_precision)
    mlflow.log_metric("test_recall", macro_recall)

    print(
        f"Test Loss: {avg_loss:.4f}, "
        f"Test Accuracy: {accuracy:.4f}, "
        f"Test F1: {f1_res} (marcro = {macro_f1:.4f}), "
        f"Test Precision: {precision_res} (marcro = {macro_precision:.4f}), "
        f"Test Recall: {recall_res} (marcro = {macro_recall:.4f}), "
    )

In [7]:
def train(
    creds: dict,
    model: nn.Module,
    save_path: str,
    remote_models_path: str,
    best_model_path: str,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: DataLoader,
    batch_size: int,
    num_classes: int,
    lr=1e-3,
    epochs=30,
    patience=5,
    resume: bool = False,
    reset_epoch: bool = False,
) -> None:
    dev_name = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device {dev_name}")
    device = torch.device(dev_name)

    # Define optimizer and loss function
    # Train only the classifier
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Checkpoint Path
    checkpoint_path = f"checkpoint_{patience}.pth"

    checkpoint_send_interval = 5

    # Track best loss for model saving
    # Load Checkpoint (Decide if you want to continue)
    start_epoch, best_val_accuracy = load_checkpoint(
        model,
        optimizer,
        checkpoint_path,
        resume,
        reset_epoch
    )

    patience_counter = 0

    f1 = MulticlassF1Score(num_classes, average=None).to(device)
    precision = MulticlassPrecision(num_classes, average=None).to(device)
    recall = MulticlassRecall(num_classes, average=None).to(device)

    # Training loop
    for epoch in range(start_epoch, epochs):
        model.train()
        epoch_loss = 0

        train_accuracy = 0

        for batch in tqdm(
            train_loader, desc=f"Epoch {epoch+1}", leave=False
        ):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            num_metadata = batch["num_metadata"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, num_metadata)
            loss = criterion(
                outputs, labels
            )  # można spróbować to logować jako osobny wykres do debugowania
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs, dim=-1)
            train_accuracy += (preds == labels).sum().item()

            f1.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)

        avg_loss = epoch_loss / len(train_loader)
        avg_train_accuracy = train_accuracy / len(train_loader.dataset)
        mlflow.log_metric("train_loss", avg_loss, step=epoch)
        mlflow.log_metric("train_acc", avg_train_accuracy, step=epoch)

        f1_res = f1.compute()
        precision_res = precision.compute()
        recall_res = recall.compute()

        for i in range(num_classes):
            mlflow.log_metric(
                f"train_f1_{ids2labels[i]}", f1_res[i], step=epoch
            )
            mlflow.log_metric(
                f"train_precision_{ids2labels[i]}",
                precision_res[i],
                step=epoch,
            )
            mlflow.log_metric(
                f"train_recall_{ids2labels[i]}", recall_res[i], step=epoch
            )

        macro_f1 = f1_res.mean()
        macro_precision = precision_res.mean()
        macro_recall = recall_res.mean()

        mlflow.log_metric("train_f1", macro_f1, step=epoch)
        mlflow.log_metric("train_precision", macro_precision, step=epoch)
        mlflow.log_metric("train_recall", macro_recall, step=epoch)

        tqdm.write(
            f"Epoch {epoch+1}: "
            f"Training Loss: {avg_loss}, "
            f"Training Accuracy: {avg_train_accuracy}, "
            f"Training F1: {macro_f1}, "
            f"Training Precision: {macro_precision}, "
            f"Training Recall: {macro_recall}"
        )

        # Validation step
        model.eval()  # Switch to evaluation mode
        val_loss = 0
        val_accuracy = 0

        f1.reset()
        precision.reset()
        recall.reset()

        with torch.no_grad():
            for batch in tqdm(
                val_loader,
                desc=f"Validation of epoch {epoch + 1}",
                leave=False,
            ):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                num_metadata = batch["num_metadata"].to(device)
                labels = batch["label"].to(device)

                outputs = model(input_ids, attention_mask, num_metadata)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                preds = torch.argmax(outputs, dim=-1)
                val_accuracy += (preds == labels).sum().item()
                f1.update(preds, labels)
                precision.update(preds, labels)
                recall.update(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = val_accuracy / len(val_loader.dataset)
        mlflow.log_metric("val_loss", avg_val_loss, step=epoch)
        mlflow.log_metric("val_acc", avg_val_accuracy, step=epoch)

        f1_res = f1.compute()
        precision_res = precision.compute()
        recall_res = recall.compute()

        for i in range(num_classes):
            mlflow.log_metric(
                f"val_f1_{ids2labels[i]}", f1_res[i], step=epoch
            )
            mlflow.log_metric(
                f"val_precision_{ids2labels[i]}",
                precision_res[i],
                step=epoch,
            )
            mlflow.log_metric(
                f"val_recall_{ids2labels[i]}", recall_res[i], step=epoch
            )

        macro_f1 = f1_res.mean()
        macro_precision = precision_res.mean()
        macro_recall = recall_res.mean()

        mlflow.log_metric("val_f1", macro_f1, step=epoch)
        mlflow.log_metric("val_precision", macro_precision, step=epoch)
        mlflow.log_metric("val_recall", macro_recall, step=epoch)

        print(
            f"Epoch {epoch+1}: "
            f"Validation Loss: {avg_val_loss}, "
            f"Validation Accuracy: {avg_val_accuracy}, "
            f"Validation F1: {macro_f1}, "
            f"Validation Precision: {macro_precision}, "
            f"Validation Recall: {macro_recall}"
        )

        save_checkpoint(
            model, optimizer, epoch, avg_val_accuracy, checkpoint_path
        )
        #if (epoch + 1) % checkpoint_send_interval == 0:# and epoch != 0:
        #    save_model_remotely(checkpoint_path, remote_models_path, creds)

        # Check for early stopping
        if avg_val_accuracy > best_val_accuracy:
            best_val_accuracy = avg_val_accuracy
            patience_counter = 0
            # Save the best model
            save_best_model(
                model,
                optimizer,
                epoch,
                best_val_accuracy,
                best_model_path
            )
            save_model_remotely(best_model_path, remote_models_path, creds)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Log final checkpoint
    save_model_remotely(checkpoint_path, remote_models_path, creds)

In [8]:
mlflow_uri = "http://cimmerian.win:5000"
resume = False
reset_epoch = False

creds = {
    'hostname': "cimmerian.win",
    'port': 22,
    'username': "",
    'password': ""
}

mlflow.set_tracking_uri(uri=mlflow_uri)

# MLflow experiment setup
mlflow.set_experiment("LiarPlusSingleRoBERTaWithDropoutClassifier")

# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaModel.from_pretrained("roberta-base")

for param in roberta.parameters():
    param.requires_grad = False  # Freeze all layers

# Hyperparameters
num_classes = 6
lr = 1e-3
epochs = 30
hidden_size = 128
# Number of epochs to wait before stopping if no improvement
patience = 10

# Save path
save_path = "/kaggle/working"
# Remote models path
remote_models_path = "/home/conan/models/single_roberta_with_dropout/"
# Best model path
best_model_path = f"{save_path}/best_model_{patience}.pth"

# można przetestować zachłannie
# dodajemy kolumnę jak poprawia i nie dodajemy jak nie poprawia
text_columns = [
    "subject",
    "speaker",
    "job_title",
    "state",
    "party_affiliation",
    "context",
]
num_metadata_cols = [
    "barely_true_counts",
    "false_counts",
    "half_true_counts",
    "mostly_true_counts",
    "pants_on_fire_counts",
]

#subset_size = 1000
#random_state = 42

# speedup the experiments
# można ustawić epochs na 1 i sprawdzić czy w ramach jednej epoki val loss spada
training_data = LiarPlusSingleRobertaDataset(
    "/kaggle/input/liar-plus-normalized-2/train2.csv",
    tokenizer,
    text_columns,
    num_metadata_cols
)
validation_data = LiarPlusSingleRobertaDataset(
    "/kaggle/input/liar-plus-normalized-2/val2.csv",
    tokenizer,
    text_columns,
    num_metadata_cols,
)
test_data = LiarPlusSingleRobertaDataset(
    "/kaggle/input/liar-plus-normalized-2/test2.csv",
    tokenizer,
    text_columns,
    num_metadata_cols,
)

batch_size = 64

#training_data_subset = Subset(training_data, sample(range(len(training_data)), k=1000))

train_dataloader = DataLoader(
    training_data, batch_size=batch_size, shuffle=True
)
val_dataloader = DataLoader(
    validation_data, batch_size=batch_size, shuffle=True
)
test_dataloader = DataLoader(
    test_data, batch_size=batch_size, shuffle=True
)

# Instantiate model
model = LiarPlusSingleRoBERTasClassifier(
    roberta,
    len(num_metadata_cols),
    hidden_size,
    num_classes,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

start = time.time()
with mlflow.start_run():
    mlflow.log_param("learning_rate", lr)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("resume", resume)
    mlflow.log_param("reset_epoch", reset_epoch)
    mlflow.log_param("patience", patience)
    
    # Train the model
    train(
        creds,
        model,
        save_path,
        remote_models_path,
        best_model_path,
        train_dataloader,
        val_dataloader,
        test_dataloader,
        batch_size,
        num_classes,
        lr,
        epochs,
        patience,
        resume,
        reset_epoch,
    )
    # Evaluate on test dataset
    test(model, best_model_path, test_dataloader)
end = time.time()
print(f"Total time took training: {end-start}s")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Epoch 23: Validation Loss: 1.7151011398860387, Validation Accuracy: 0.21573208722741433, Validation F1: 0.14310072362422943, Validation Precision: 0.24014638364315033, Validation Recall: 0.19755536317825317
Checkpoint saved at epoch 22 with validation accuracy 0.2157
Early stopping at epoch 23


Uploading checkpoint_10.pth: 100%|██████████| 410k/410k [00:03<00:00, 133kB/s]


Plik checkpoint_10.pth został wysłany.
Model loaded from best model checkpoint.


Evaluating: 100%|██████████| 21/21 [00:23<00:00,  1.13s/it]


Test Loss: 1.7257, Test Accuracy: 0.2190, Test F1: tensor([0.1143, 0.1257, 0.0000, 0.3133, 0.2693, 0.0000], device='cuda:0') (marcro = 0.1371), Test Precision: tensor([0.4615, 0.2200, 0.0000, 0.2024, 0.2656, 0.0000], device='cuda:0') (marcro = 0.1916), Test Recall: tensor([0.0652, 0.0880, 0.0000, 0.6929, 0.2731, 0.0000], device='cuda:0') (marcro = 0.1865), 
🏃 View run marvelous-wolf-714 at: http://cimmerian.win:5000/#/experiments/5/runs/389d1f463530440f970538495c82f46f
🧪 View experiment at: http://cimmerian.win:5000/#/experiments/5
Total time took training: 6404.202946901321s
